## 本番データで料理分類器を作ってみる

In [26]:
import pandas as pd
# from io import StringIO

# """https://stackoverflow.com/questions/19611729/getting-google-spreadsheet-csv-into-a-pandas-dataframe
# """
# import requests
# url = 'https://docs.google.com/spreadsheets/d/1a4fkAJNessYsmscETtugik3VtDf2fNzdToTeAUuhrYA/export?format=csv&id=cuisine'
# data = requests.get(url).content
# pd.read_csv(StringIO(data))
# pd.read_csv('https://docs.google.com/spreadsheets/d/1a4fkAJNessYsmscETtugik3VtDf2fNzdToTeAUuhrYA/export?format=csv')

cuisine_df = pd.read_csv('chefgohan.tsv', sep='\t').drop(['who'], axis=1).set_index('cuisine').query('is_standard == True').fillna('')
cuisine_df.head()

,genre,purpose,hotorcold,taste,materials,when,color,is_standard,url
cuisine,,,,,,,,,
親子丼,japanese,main,hot,"salty,sweet","rice,meat","lunch,dinner","yellow,green,white",True,https://c-chefgohan.gnst.jp/imgdata/recipe/89/...
玉子焼き,japanese,side,hot,"salty,sweet",,breakfast,yellow,True,https://c-chefgohan.gnst.jp/imgdata/recipe/90/...
鶏のから揚げ,japanese,side,hot,salty,meat,"lunch,dinner",brown,True,https://c-chefgohan.gnst.jp/imgdata/recipe/91/...
肉じゃが,japanese,side,hot,"salty,sweet","meat,vegetable",dinner,"brown,green,red",True,https://c-chefgohan.gnst.jp/imgdata/recipe/95/...
天ぷら,japanese,side,hot,salty,"fish,vegetable","lunch,dinner","yellow,red,green",True,https://c-chefgohan.gnst.jp/imgdata/recipe/96/...


In [2]:
# import pandas as pd

# cuisine = [
#     {
#        'name': 'remen',
#         'genre': 'chinese',
#         'taste': ['spicy'],
#         'hotorcold': 'cold',
#     },
#     {
#        'name': 'baconeggs',
#         'genre': 'western',
#         'taste': ['salty'],
#         'hotorcold': 'hot',
#     },
#     {
#        'name': 'ebichili',
#         'genre': 'chinese',
#         'taste': ['spicy'],
#         'hotorcold': 'hot',
#     },
#     {
#        'name': 'oyakodon',
#         'genre': 'japanese',
#         'taste': ['salty', 'sweety'],
#         'hotorcold': 'hot',
#     },
# ]
# cuisine_df = pd.DataFrame(cuisine)
# cuisine_df.set_index('name')

,genre,hotorcold,taste
name,,,
remen,chinese,cold,[spicy]
baconeggs,western,hot,[salty]
ebichili,chinese,hot,[spicy]
oyakodon,japanese,hot,"[salty, sweety]"


### 料理データで決定木

### Preprocessing

In [3]:
"""nameの分類をしたいので、name以外のカラムを数値化していく（学習させるため）
カテゴリ変数->数値化は基本的にlabelbinarizerでok...
と思いきや決定系はnumericかbinarizeがいいようだ
https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931
"""

print(cuisine_df.columns)

cuisine_df.tail()

Index(['genre', 'purpose', 'hotorcold', 'taste', 'materials', 'when', 'color',
       'is_standard', 'url'],
      dtype='object')


,genre,purpose,hotorcold,taste,materials,when,color,is_standard,url
cuisine,,,,,,,,,
チョコチップクッキー,western,dessert,cold,sweet,,snack,"black,brown",True,https://c-chefgohan.gnst.jp/imgdata/recipe/22/...
イチゴのレアチーズケーキ,western,dessert,cold,"sweet,sour",,snack,"white,red",True,https://c-chefgohan.gnst.jp/imgdata/recipe/69/...
アップルパイ,western,dessert,cold,"sweet,sour",,snack,"brown,red",True,https://c-chefgohan.gnst.jp/imgdata/recipe/66/...
みたらし団子,japanese,dessert,cold,"sweet,salty",,snack,"brown,white",True,https://c-chefgohan.gnst.jp/imgdata/recipe/55/...
イチゴのショートケーキ,western,dessert,cold,"sweet,sour",,"snack,dinner","white,red,yellow",True,https://c-chefgohan.gnst.jp/imgdata/recipe/77/...


### genre

In [4]:
cuisine_df.genre.unique()

array(['japanese', 'western', 'chinese', 'french', 'italian', 'spanish',
       'asian'], dtype=object)

In [5]:
# from sklearn.preprocessing import LabelEncoder
# le_genre = LabelEncoder()
# labeled_genre = le_genre.fit_transform(cuisine_df.genre)
# labeled_genre, le_genre.inverse_transform(labeled_genre)

(array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 5,
        5, 0, 0, 0, 0, 1, 6, 6, 6, 6, 4, 6]),
 array(['japanese', 'japanese', 'japanese', 'japanese', 'japanese',
        'japanese', 'japanese', 'japanese', 'japanese', 'japanese',
        'japanese', 'japanese', 'japanese', 'japanese', 'japanese',
        'japanese', 'japanese', 'japanese', 'japanese', 'japanese',
        'japanese', 'japanese', 'japanese', 'japanese', 'japanese',
        'japanese', 'japanese', 'japanese', 'western', 'western', 'western',
        'western', 'western', 'western', 'western', 'western', 'western',
        'western', 'western', 'western', 'western', 'chinese', 'chinese',
        'chinese', 'chinese', 'chinese', 'chinese', 'chinese', 'chinese',
        'chinese', 'chinese', 'chinese', 'chinese', 'chinese', 'chinese',
        'c

In [3]:
"""str -> array
"""
cuisine_df['parsed_genre'] = cuisine_df.genre.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_genre'].head()

cuisine
親子丼       [japanese]
玉子焼き      [japanese]
鶏のから揚げ    [japanese]
肉じゃが      [japanese]
天ぷら       [japanese]
Name: parsed_genre, dtype: object

In [4]:
"""このカラムは配列ではないがone hotの方がT/Fで扱いやすいのでMultiLaberBinarizerにしてみる
"""

from sklearn.preprocessing import MultiLabelBinarizer
lb_genre = MultiLabelBinarizer()
labeled_genre = lb_genre.fit_transform(cuisine_df.parsed_genre.values)
labeled_genre, lb_genre.inverse_transform(labeled_genre)

(array([[0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        

### purpose

In [6]:
# from sklearn.preprocessing import LabelEncoder
# le_purpose = LabelEncoder()
# labeled_purpose = le_purpose.fit_transform(cuisine_df.purpose)
# labeled_purpose, le_purpose.inverse_transform(labeled_purpose)

(array([1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 1, 1, 3, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1,
        2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1,
        2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0]),
 array(['main', 'side', 'side', 'side', 'side', 'side', 'side', 'main',
        'main', 'main', 'side', 'side', 'side', 'side', 'side', 'main',
        'side', 'side', 'side', 'side', 'side', 'side', 'side', 'side',
        'side', 'main', 'main', 'sidle', 'main', 'side', 'side', 'side',
        'side', 'side', 'main', 'main', 'main', 'side', 'side', 'side',
        'side', 'side', 'side', 'side', 'side', 'main', 'side', 'main',
        'side', 'side', 'main', 'side', 'side', 'side', 'side', 'main',
        'side', 'side', 'side', 'side', 'side', 'side', 'side', 'side',
        'main', 'main', 'side', 'main', 'main', 'side', 'main', 'side',
        'side', 'side', 'dessert', 'dessert', 'dessert', 'dessert',
      

In [27]:
"""str -> array
"""
cuisine_df['parsed_purpose'] = cuisine_df.purpose.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_purpose'].head()

cuisine
親子丼       [main]
玉子焼き      [side]
鶏のから揚げ    [side]
肉じゃが      [side]
天ぷら       [side]
Name: parsed_purpose, dtype: object

In [28]:
"""このカラムは配列ではないがone hotの方がT/Fで扱いやすいのでMultiLaberBinarizerにしてみる
"""

from sklearn.preprocessing import MultiLabelBinarizer
lb_purpose = MultiLabelBinarizer()
labeled_purpose = lb_purpose.fit_transform(cuisine_df.parsed_purpose.values)
labeled_purpose, lb_purpose.inverse_transform(labeled_purpose)

(array([[0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 

### hot or cold

In [10]:
from sklearn.preprocessing import LabelEncoder
le_hotorcold = LabelEncoder()
labeled_hotorcold = le_hotorcold.fit_transform(cuisine_df.hotorcold)
labeled_hotorcold, le_hotorcold.inverse_transform(labeled_hotorcold)

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array(['hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot',
        'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot',
        'cold', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot',
        'cold', 'hot', 'hot', 'cold', 'cold', 'hot', 'hot', 'hot', 'hot',
        'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot',
        'hot', 'hot', 'hot', 'hot', 'cold', 'cold', 'hot', 'hot', 'hot',
        'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'cold',
        'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot', 'hot',
        'cold', 'cold', 'cold', 'cold', 'cold', 'cold', 'cold', 'cold',
        'cold'], dtype=object))

### taste

In [11]:
# """str -> array; column: taste 
# 順番だけ整列させて文字列に戻す
# """
# cuisine_df['parsed_taste'] = cuisine_df.taste.apply(lambda t: sorted(t.split(','))).apply(','.join)
# cuisine_df.taste.head(), cuisine_df.parsed_taste.head()

"""str -> array
順番も整列化
"""
cuisine_df['parsed_taste'] = cuisine_df.taste.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_taste'].head()

cuisine
親子丼       [salty, sweet]
玉子焼き      [salty, sweet]
鶏のから揚げ           [salty]
肉じゃが      [salty, sweet]
天ぷら              [salty]
Name: parsed_taste, dtype: object

In [12]:
"""このカラムは配列なのでMultiLaberBinarizerでないといけない
"""

from sklearn.preprocessing import MultiLabelBinarizer
lb_taste = MultiLabelBinarizer()
labeled_taste = lb_taste.fit_transform(cuisine_df.parsed_taste.values)
labeled_taste, lb_taste.inverse_transform(labeled_taste)

# le_taste = LabelEncoder()
# labeled_taste = le_taste.fit_transform(cuisine_df.parsed_taste)
# labeled_taste, le_taste.inverse_transform(labeled_taste)

(array([[0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [1, 1, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],


### materials

In [13]:
# """str -> array; column: taste 
# 順番だけ整列させて文字列に戻す
# """
# cuisine_df['parsed_materials'] = cuisine_df.materials.apply(lambda t: sorted(t.split(','))).apply(','.join)
# cuisine_df.materials.head(), cuisine_df.parsed_materials.head()

"""str -> array
順番も整列化
"""
cuisine_df['parsed_materials'] = cuisine_df.materials.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_materials'].head()

cuisine
親子丼            [meat, rice]
玉子焼き                     []
鶏のから揚げ               [meat]
肉じゃが      [meat, vegetable]
天ぷら       [fish, vegetable]
Name: parsed_materials, dtype: object

In [14]:
"""このカラムは配列なのでMultiLaberBinarizerでないといけない
"""
lb_materials = MultiLabelBinarizer()
labeled_materials = lb_materials.fit_transform(cuisine_df.parsed_materials.values)
labeled_materials, lb_materials.inverse_transform(labeled_materials)

# le_materials = LabelEncoder()
# labeled_materials = le_materials.fit_transform(cuisine_df.parsed_materials)
# labeled_materials, le_materials.inverse_transform(labeled_materials)

(array([[0, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0,

### when

In [15]:
# """str -> array; column: taste 
# 順番だけ整列させて文字列に戻す
# """

# cuisine_df['parsed_when'] = cuisine_df.when.apply(lambda t: sorted(t.split(','))).apply(','.join)
# cuisine_df.when.head(), cuisine_df.parsed_when.head()


"""str -> array
順番も整列化
"""
cuisine_df['parsed_when'] = cuisine_df.when.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_when'].head()

cuisine
親子丼       [dinner, lunch]
玉子焼き          [breakfast]
鶏のから揚げ    [dinner, lunch]
肉じゃが             [dinner]
天ぷら       [dinner, lunch]
Name: parsed_when, dtype: object

In [16]:
# """もしかしたらMultiLaberBinarizerのほうがいいかもしれない
# """
# le_when = LabelEncoder()
# labeled_when = le_when.fit_transform(cuisine_df.parsed_when)
# labeled_when, le_when.inverse_transform(labeled_when)


"""MultiLabelBinarizerにしてみた
"""

lb_when = MultiLabelBinarizer()
labeled_when = lb_when.fit_transform(cuisine_df.parsed_when.values)
labeled_when, lb_when.inverse_transform(labeled_when)

(array([[0, 1, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [1, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0

### color

In [17]:
# """str -> array; column: taste 
# 順番だけ整列させて文字列に戻す
# """

# cuisine_df['parsed_color'] = cuisine_df.color.apply(lambda t: sorted(t.split(','))).apply(','.join)
# cuisine_df.color.head(), cuisine_df.parsed_color.head()

"""str -> array
順番も整列化
"""
cuisine_df['parsed_color'] = cuisine_df.color.apply(lambda t: sorted(t.split(',')))
cuisine_df['parsed_color'].head()

cuisine
親子丼       [green, white, yellow]
玉子焼き                    [yellow]
鶏のから揚げ                   [brown]
肉じゃが         [brown, green, red]
天ぷら         [green, red, yellow]
Name: parsed_color, dtype: object

In [18]:
# """もしかしたらMultiLaberBinarizerのほうがいいかもしれない
# """
# le_color = LabelEncoder()
# labeled_color = le_color.fit_transform(cuisine_df.parsed_color)
# labeled_color, le_color.inverse_transform(labeled_color)


"""MultiLabelBinarizerにしてみた
"""

lb_color = MultiLabelBinarizer()
labeled_color = lb_color.fit_transform(cuisine_df.parsed_color.values)
labeled_color, lb_color.inverse_transform(labeled_color)

(array([[0, 0, 0, 1, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
 

### name

In [19]:
"""最後に目的変数の料理名をID化
"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(cuisine_df.index)
y, le.inverse_transform(y)

(array([69, 63, 75, 67, 55,  4, 76, 48, 66,  1,  7,  9, 38,  6, 64, 62,  5,
         2, 10, 24, 71, 77, 74, 68, 56, 19,  3, 54, 20, 18, 46, 40, 41, 47,
        31, 17, 30, 22, 25, 43, 36, 16, 72,  0,  8, 27, 73, 53, 70, 59, 50,
        57, 51, 80, 52, 61, 78, 79, 49, 42, 35, 44, 26, 37, 39, 23, 12, 45,
        32, 60, 34, 29, 21, 65, 58, 33, 28, 15, 13, 11, 14]),
 array(['親子丼', '玉子焼き', '鶏のから揚げ', '肉じゃが', '天ぷら', 'きんぴら', '鶏の照り焼き',
        '五目炊き込みごはん', '筍ごはん', 'おでん', 'すき焼き', 'ひじきの煮物', 'ブリ大根', 'さばの味噌煮',
        '生姜焼き', '牛丼', 'さといもの煮ころがし', 'かぼちゃの煮物', 'ほうれん草のごまあえ', 'カレイの煮付け',
        '豚汁', '鶏の竜田揚げ', '鯵の南蛮漬け', '西京焼き', '揚げ出し豆腐', 'エビ天そば', 'きつねそば',
        '大根サラダ', 'オムライス', 'エビフライ', 'ローストビーフ', 'ポテトサラダ', 'マカロニグラタン',
        'ロールキャベツ', 'ナポリタン', 'エビピラフ', 'ドライカレー', 'カニクリームコロッケ', 'クリームシチュー',
        'メンチカツ', 'フライドチキン', 'エビチリ', '酢豚', 'あんかけ かに玉', 'にらたま', 'チャーハン',
        '青椒肉絲（チンジャオロース）', '坦々麺', '豚の角煮', '棒々鶏 （バンバンジー)', '冷やし中華', '春巻き',
        '卵スープ', '麻婆豆腐', '回鍋肉 （ホイコーロー）', '焼きそば', '麻婆春雨', '麻婆茄子', '八宝菜'

## concatenate labeled data

In [29]:
import numpy as np

# X = np.concatenate((
#     labeled_genre.reshape(-1, 1),
#     labeled_purpose.reshape(-1, 1),
#     labeled_hotorcold.reshape(-1, 1),
#     labeled_taste.reshape(-1, 1),
#     labeled_materials.reshape(-1, 1),
#     labeled_when.reshape(-1, 1),
#     labeled_color.reshape(-1, 1),
# ), axis=1)
# X, y

# X = np.concatenate((
#     labeled_genre.reshape(-1, 1),
#     labeled_purpose.reshape(-1, 1),
#     labeled_hotorcold.reshape(-1, 1),
#     labeled_taste,
#     labeled_materials,
#     labeled_when,
#     labeled_color,
# ), axis=1)
# X, y

X = np.concatenate((
    labeled_genre,
    labeled_purpose,
    labeled_hotorcold.reshape(-1, 1),
    labeled_taste,
    labeled_materials,
    labeled_when,
    labeled_color,
), axis=1)
X, y

(array([[0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 1, 1, 1]]),
 array([69, 63, 75, 67, 55,  4, 76, 48, 66,  1,  7,  9, 38,  6, 64, 62,  5,
         2, 10, 24, 71, 77, 74, 68, 56, 19,  3, 54, 20, 18, 46, 40, 41, 47,
        31, 17, 30, 22, 25, 43, 36, 16, 72,  0,  8, 27, 73, 53, 70, 59, 50,
        57, 51, 80, 52, 61, 78, 79, 49, 42, 35, 44, 26, 37, 39, 23, 12, 45,
        32, 60, 34, 29, 21, 65, 58, 33, 28, 15, 13, 11, 14]))

## Training

In [30]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
"""モデルのセーブ
"""
from datetime import datetime
from sklearn.externals import joblib

def _save_model(_model, localpath):
    joblib.dump(_model, localpath)
    return localpath

# def _model_path():
#     return '../static/{}_{}.pkl.gz'.format(datetime.now().strftime('%Y%m%d'), 'cuisines')

# _save_model(model, _model_path())
_save_model(model, 'model_cuisines.pkl.gz')

'model_cuisines.pkl.gz'

In [32]:
def create_feature_name(column_name):
    """カラム名->数値化したカラム名に変換
    """
    labels = eval('lb_{}'.format(column_name)).classes_
    if len(labels) <= 2:
        return ['{}__'.format(column_name)]
    else:
        return ['{}__{}'.format(column_name, label) for label in labels]

create_feature_name('taste') #,　create_feature_name('hotorcold')

['taste__bitter',
 'taste__salty',
 'taste__sour',
 'taste__spicy',
 'taste__sweet']

In [33]:
from itertools import chain

def flatten(l):
    return list(chain.from_iterable(l))

def create_feature_columns():
#     return flatten([create_feature_name(name) for name in ['genre', 'hotorcold', 'taste']])
    return flatten([create_feature_name(name) for name in ['genre', 'purpose', 'taste', 'materials', 'when', 'color']])

create_feature_columns()

['genre__asian',
 'genre__chinese',
 'genre__french',
 'genre__italian',
 'genre__japanese',
 'genre__spanish',
 'genre__western',
 'purpose__dessert',
 'purpose__main',
 'purpose__side',
 'taste__bitter',
 'taste__salty',
 'taste__sour',
 'taste__spicy',
 'taste__sweet',
 'materials__',
 'materials__fish',
 'materials__meat',
 'materials__noodle',
 'materials__rice',
 'materials__vegetable',
 'when__breakfast',
 'when__dinner',
 'when__lunch',
 'when__snack',
 'color__black',
 'color__brown',
 'color__gray',
 'color__green',
 'color__orange',
 'color__pink',
 'color__red',
 'color__white',
 'color__yellow']

In [25]:
le.inverse_transform(y)

array(['親子丼', '玉子焼き', '鶏のから揚げ', '肉じゃが', '天ぷら', 'きんぴら', '鶏の照り焼き',
       '五目炊き込みごはん', '筍ごはん', 'おでん', 'すき焼き', 'ひじきの煮物', 'ブリ大根', 'さばの味噌煮',
       '生姜焼き', '牛丼', 'さといもの煮ころがし', 'かぼちゃの煮物', 'ほうれん草のごまあえ', 'カレイの煮付け',
       '豚汁', '鶏の竜田揚げ', '鯵の南蛮漬け', '西京焼き', '揚げ出し豆腐', 'エビ天そば', 'きつねそば',
       '大根サラダ', 'オムライス', 'エビフライ', 'ローストビーフ', 'ポテトサラダ', 'マカロニグラタン',
       'ロールキャベツ', 'ナポリタン', 'エビピラフ', 'ドライカレー', 'カニクリームコロッケ', 'クリームシチュー',
       'メンチカツ', 'フライドチキン', 'エビチリ', '酢豚', 'あんかけ かに玉', 'にらたま', 'チャーハン',
       '青椒肉絲（チンジャオロース）', '坦々麺', '豚の角煮', '棒々鶏 （バンバンジー)', '冷やし中華', '春巻き',
       '卵スープ', '麻婆豆腐', '回鍋肉 （ホイコーロー）', '焼きそば', '麻婆春雨', '麻婆茄子', '八宝菜',
       'ムニエル', 'ビーフシチュー', 'ラタトゥイユ', 'シーザーサラダ', 'ブイヤベース', 'ペペロンチーノ',
       'カルボナーラ', 'アクアパッツァ', 'リゾット', 'パエリア', '海老のアヒージョ', 'ビビンパ', 'トムヤムクン',
       'カクテキ（大根キムチ）', '生春巻き', '杏仁豆腐', 'パンナコッタ', 'チョコチップクッキー',
       'イチゴのレアチーズケーキ', 'アップルパイ', 'みたらし団子', 'イチゴのショートケーキ'], dtype=object)

In [36]:
# %matplotlib inline
# from sklearn.externals.six import StringIO
# from IPython.display import Image
# import pydot_ng

# dot_data = StringIO()
# export_graphviz(model, 
#                 out_file=dot_data, 
#                 feature_names=['genre', 'purpose', 'hotorcold'] + create_feature_columns(),
#                 class_names=le.inverse_transform(y),
#                 filled=True,
#                 rounded=True,
#                 proportion=True,
#                )
# graph = pydot_ng.graph_from_dot_data(dot_data.getvalue())
# Image(graph.create_png())

In [202]:
genre_dict = pd.DataFrame({
    'id': labeled_genre,
    'name': le_genre.inverse_transform(labeled_genre),
}).drop_duplicates().to_dict(orient='records')
genre_dict

[{'id': 4, 'name': 'japanese'},
 {'id': 6, 'name': 'western'},
 {'id': 1, 'name': 'chinese'},
 {'id': 2, 'name': 'french'},
 {'id': 3, 'name': 'italian'},
 {'id': 5, 'name': 'spanish'},
 {'id': 0, 'name': 'asian'}]

In [203]:
purpose_dict = pd.DataFrame({
    'id': labeled_purpose,
    'name': le_purpose.inverse_transform(labeled_purpose),
}).drop_duplicates().to_dict(orient='records')
purpose_dict

[{'id': 1, 'name': 'main'},
 {'id': 2, 'name': 'side'},
 {'id': 3, 'name': 'sidle'},
 {'id': 0, 'name': 'dessert'}]

In [204]:
hotorcold_dict = pd.DataFrame({
    'id': labeled_hotorcold,
    'name': le_hotorcold.inverse_transform(labeled_hotorcold),
}).drop_duplicates().to_dict(orient='records')
hotorcold_dict

[{'id': 1, 'name': 'hot'}, {'id': 0, 'name': 'cold'}]

In [209]:
# taste_dict = pd.DataFrame({
#     'id': labeled_taste,
#     'name': lb_taste.inverse_transform(labeled_taste),
# }).drop_duplicates().to_dict(orient='records')
# taste_dict

In [210]:
# materials_dict = pd.DataFrame({
#     'id': labeled_materials,
#     'name': le_materials.inverse_transform(labeled_materials),
# }).drop_duplicates().to_dict(orient='records')
# materials_dict

In [211]:
# when_dict = pd.DataFrame({
#     'id': labeled_when,
#     'name': le_when.inverse_transform(labeled_when),
# }).drop_duplicates().sort_values('id').to_dict(orient='records')
# when_dict

In [212]:
# color_dict = pd.DataFrame({
#     'id': labeled_color,
#     'name': le_color.inverse_transform(labeled_color),
# }).drop_duplicates().to_dict(orient='records')
# color_dict

In [128]:
y_dict = pd.DataFrame({
    'id': y,
    'name': le.inverse_transform(y),
}).drop_duplicates().to_dict(orient='records')
y_dict

[{'id': 69, 'name': '親子丼'},
 {'id': 63, 'name': '玉子焼き'},
 {'id': 75, 'name': '鶏のから揚げ'},
 {'id': 67, 'name': '肉じゃが'},
 {'id': 55, 'name': '天ぷら'},
 {'id': 4, 'name': 'きんぴら'},
 {'id': 76, 'name': '鶏の照り焼き'},
 {'id': 48, 'name': '五目炊き込みごはん'},
 {'id': 66, 'name': '筍ごはん'},
 {'id': 1, 'name': 'おでん'},
 {'id': 7, 'name': 'すき焼き'},
 {'id': 9, 'name': 'ひじきの煮物'},
 {'id': 38, 'name': 'ブリ大根'},
 {'id': 6, 'name': 'さばの味噌煮'},
 {'id': 64, 'name': '生姜焼き'},
 {'id': 62, 'name': '牛丼'},
 {'id': 5, 'name': 'さといもの煮ころがし'},
 {'id': 2, 'name': 'かぼちゃの煮物'},
 {'id': 10, 'name': 'ほうれん草のごまあえ'},
 {'id': 24, 'name': 'カレイの煮付け'},
 {'id': 71, 'name': '豚汁'},
 {'id': 77, 'name': '鶏の竜田揚げ'},
 {'id': 74, 'name': '鯵の南蛮漬け'},
 {'id': 68, 'name': '西京焼き'},
 {'id': 56, 'name': '揚げ出し豆腐'},
 {'id': 19, 'name': 'エビ天そば'},
 {'id': 3, 'name': 'きつねそば'},
 {'id': 54, 'name': '大根サラダ'},
 {'id': 20, 'name': 'オムライス'},
 {'id': 18, 'name': 'エビフライ'},
 {'id': 46, 'name': 'ローストビーフ'},
 {'id': 40, 'name': 'ポテトサラダ'},
 {'id': 41, 'name': 'マカロニグラタン'},
 {'id':

In [129]:
genre_dict, purpose_dict, hotorcold_dict, taste_dict, materials_dict, when_dict, color_dict, y_dict

([{'id': 4, 'name': 'japanese'},
  {'id': 6, 'name': 'western'},
  {'id': 1, 'name': 'chinese'},
  {'id': 2, 'name': 'french'},
  {'id': 3, 'name': 'italian'},
  {'id': 5, 'name': 'spanish'},
  {'id': 0, 'name': 'asian'}],
 [{'id': 1, 'name': 'main'},
  {'id': 2, 'name': 'side'},
  {'id': 3, 'name': 'sidle'},
  {'id': 0, 'name': 'dessert'}],
 [{'id': 1, 'name': 'hot'}, {'id': 0, 'name': 'cold'}],
 [{'id': 8, 'name': 'salty,sweet'},
  {'id': 4, 'name': 'salty'},
  {'id': 3, 'name': 'saily,sweet'},
  {'id': 6, 'name': 'salty,spicy'},
  {'id': 0, 'name': 'bitter,daily,salty'},
  {'id': 14, 'name': 'sweet'},
  {'id': 1, 'name': 'bitter,salty,sweet'},
  {'id': 10, 'name': 'sour,spicy'},
  {'id': 2, 'name': 'saily,sour,spicy,sweet'},
  {'id': 7, 'name': 'salty,spicy,sweet'},
  {'id': 5, 'name': 'salty,sour,sweet'},
  {'id': 11, 'name': 'sour,sweet'},
  {'id': 12, 'name': 'spicy'},
  {'id': 9, 'name': 'sour'},
  {'id': 13, 'name': 'spicy,sweet'}],
 [{'id': 5, 'name': 'meat,rice'},
  {'id': 0,